This notebook is a fork of https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-infer

In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns, gc

In [ ]:
dim = 512 #1024, 256, 'original'
test_dir = f'/kaggle/input/vinbigdata-{dim}-image-dataset/vinbigdata/test'

In [ ]:
test_df = pd.read_csv(f'/kaggle/input/vinbigdata-{dim}-image-dataset/vinbigdata/test.csv')
test_df.head()

# YOLOv5 Stuff

In [ ]:
shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5') # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# Inference

In [ ]:
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

In [ ]:
for i in range(1,6):
    torch.cuda.empty_cache()
    gc.collect()
    WEIGHT_FILE = f'/kaggle/input/resnet50d-096-stage-2/FOLD{i}.pt'    
    !python detect.py --weights $WEIGHT_FILE --img 512 --conf 0.15 --iou 0.5 --source $test_dir --save-txt --save-conf --exist-ok
    image_ids = []
    PredictionStrings = []

    for file_path in glob('runs/detect/exp/labels/*txt'):
        image_id = file_path.split('/')[-1].split('.')[0]
        w, h = test_df.loc[test_df.image_id==image_id,['width', 'height']].values[0]
        f = open(file_path, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        bboxes = list(np.concatenate((data[:, :2], yolo2voc(h, w, data[:, 2:])), axis =1).reshape(-1).astype(str))
        for idx in range(len(bboxes)):
            bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
        image_ids.append(image_id)
        PredictionStrings.append(' '.join(bboxes))
        os.remove(file_path)

    pred_df = pd.DataFrame({'image_id':image_ids,
                            'PredictionString':PredictionStrings})
    sub_df = pd.merge(test_df, pred_df, on = 'image_id', how = 'left').fillna("14 1 0 0 1 1")
    sub_df = sub_df[['image_id', 'PredictionString']]
    sub_df.to_csv(f'/kaggle/working/Fold_{i}_cleaned.csv',index = False)

In [ ]:
for i in range(1,6):
    torch.cuda.empty_cache()
    gc.collect()
    WEIGHT_FILE = f'/kaggle/input/resnet50d-096-stage-2/FOLD{i}.pt'    
    !python detect.py --weights $WEIGHT_FILE --img 512 --conf 0.00001 --iou 0.0 --source $test_dir --save-txt --save-conf --exist-ok
    image_ids = []
    PredictionStrings = []

    for file_path in glob('runs/detect/exp/labels/*txt'):
        image_id = file_path.split('/')[-1].split('.')[0]
        w, h = test_df.loc[test_df.image_id==image_id,['width', 'height']].values[0]
        f = open(file_path, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        bboxes = list(np.concatenate((data[:, :2], yolo2voc(h, w, data[:, 2:])), axis =1).reshape(-1).astype(str))
        for idx in range(len(bboxes)):
            bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
        image_ids.append(image_id)
        PredictionStrings.append(' '.join(bboxes))
        os.remove(file_path)

    pred_df = pd.DataFrame({'image_id':image_ids,
                            'PredictionString':PredictionStrings})
    sub_df = pd.merge(test_df, pred_df, on = 'image_id', how = 'left').fillna("14 1 0 0 1 1")
    sub_df = sub_df[['image_id', 'PredictionString']]
    sub_df.to_csv(f'/kaggle/working/Fold_{i}.csv',index = False)

In [ ]:
for i in range(1,6):
    torch.cuda.empty_cache()
    gc.collect()
    WEIGHT_FILE = f'/kaggle/input/resnet50d-096-stage-2/FOLD{i}.pt'    
    !python detect.py --weights $WEIGHT_FILE --img 512 --conf 0.15 --iou 0.5 --source $test_dir --save-txt --save-conf --exist-ok --augment
    image_ids = []
    PredictionStrings = []

    for file_path in glob('runs/detect/exp/labels/*txt'):
        image_id = file_path.split('/')[-1].split('.')[0]
        w, h = test_df.loc[test_df.image_id==image_id,['width', 'height']].values[0]
        f = open(file_path, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        bboxes = list(np.concatenate((data[:, :2], yolo2voc(h, w, data[:, 2:])), axis =1).reshape(-1).astype(str))
        for idx in range(len(bboxes)):
            bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
        image_ids.append(image_id)
        PredictionStrings.append(' '.join(bboxes))
        os.remove(file_path)

    pred_df = pd.DataFrame({'image_id':image_ids,
                            'PredictionString':PredictionStrings})
    sub_df = pd.merge(test_df, pred_df, on = 'image_id', how = 'left').fillna("14 1 0 0 1 1")
    sub_df = sub_df[['image_id', 'PredictionString']]
    sub_df.to_csv(f'/kaggle/working/Fold_{i}_TTA.csv',index = False)

In [ ]:
# torch.cuda.empty_cache()
# gc.collect()
# WEIGHT_FILE = f'/kaggle/input/resnet50d-096-stage-2/FOLD5.pt'    
# !python detect.py --weights $WEIGHT_FILE --img 512 --conf 0.0 --iou 0.0 --source $test_dir --save-txt --save-conf --exist-ok --augment
# image_ids = []
# PredictionStrings = []

# for file_path in glob('runs/detect/exp/labels/*txt'):
#     image_id = file_path.split('/')[-1].split('.')[0]
#     w, h = test_df.loc[test_df.image_id==image_id,['width', 'height']].values[0]
#     f = open(file_path, 'r')
#     data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
#     data = data[:, [0, 5, 1, 2, 3, 4]]
#     bboxes = list(np.concatenate((data[:, :2], yolo2voc(h, w, data[:, 2:])), axis =1).reshape(-1).astype(str))
#     for idx in range(len(bboxes)):
#         bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
#     image_ids.append(image_id)
#     PredictionStrings.append(' '.join(bboxes))

# pred_df = pd.DataFrame({'image_id':image_ids,
#                         'PredictionString':PredictionStrings})
# sub_df = pd.merge(test_df, pred_df, on = 'image_id', how = 'left').fillna("14 1 0 0 1 1")
# sub_df = sub_df[['image_id', 'PredictionString']]
# sub_df.to_csv(f'/kaggle/working/Fold_5_uncleaned.csv',index = False)

In [ ]:
shutil.rmtree('/kaggle/working/yolov5')